In [43]:
import pandas as pd
import numpy as np
import nltk
import json
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
nltk.download('all', force=True)
nltk.data.path.append("/Users/laurenceliao/nltk_data")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/laurenceliao/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/laurenceliao/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/laurenceliao/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/laurenceliao/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/laurenceliao/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |     /U

In [12]:
# currently testing this with random csv file from kaggle
data = pd.read_csv("monster_com-job_sample.csv")
data

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,United States of America,US,NaN,No,jobs.monster.com,This is a major premier Cincinnati based finan...,Assistant Vice President - Controller Job in C...,Full Time,"Cincinnati, OH",NaN,http://jobview.monster.com/Assistant-Vice-Pres...,"120,000.00 - 160,000.00 $ /yearbonus",NaN,a80bc8cc3a90c17eef418963803bc640
21996,United States of America,US,NaN,No,jobs.monster.com,Luxury homebuilder in Cincinnati seeking multi...,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45236",Construction - Residential & Commercial/Office,http://jobview.monster.com/Accountant-Job-Cinc...,"45,000.00 - 60,000.00 $ /year",Manager (Manager/Supervisor of Staff),419a3714be2b30a10f628de207d041de
21997,United States of America,US,NaN,No,jobs.monster.com,RE: Adobe AEM- Client - Loca...,AEM/CQ developer Job in Chicago,Full Time,"Chicago, IL 60602",NaN,http://jobview.monster.com/AEM-CQ5-developer-J...,NaN,NaN,5a590350b73b2cec46b05750a208e345
21998,United States of America,US,NaN,No,jobs.monster.com,Jernberg Industries was established in 1937 an...,Electrician - Experienced Forging Electrician ...,Full Time Employee,"Chicago, IL 60609","Jernberg Industries, Inc.",http://jobview.monster.com/Electrician-Experie...,25.00 - 28.00 $ /hour,Installation/Maintenance/Repair,40161cf61c283af9dc2b0a62947a5f1b


In [291]:
tech_data = pd.read_csv("tech_jobs_trainingset.csv")

tech_data

,advertiserurl,jobdescription,jobtitle,skills,tag,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,Looking for Selenium engineers...must have sol...,AUTOMATION TEST ENGINEER,SEE BELOW,automation,NaN,Buckets:,tag: \n-automation\n-it\n-backend_dev...
1,https://www.dice.com/jobs/detail/Information-S...,The University of Chicago has a rapidly growin...,Information Security Engineer,"linux/unix, network monitoring, incident respo...",it,NaN,DevOps\nPM (project management)\nTesting\nmain...,qa
2,https://www.dice.com/jobs/detail/Business-Solu...,"GalaxE.SolutionsEvery day, our solutions affec...",Business Solutions Architect,"Enterprise Solutions Architecture, business in...",architect,NaN,NaN,bi
3,https://www.dice.com/jobs/detail/Java-Develope...,Java DeveloperFull-time/direct-hireBolingbrook...,"Java Developer (mid level)- FT- GREAT culture,...",Please see job description,backend_dev,NaN,NaN,architect
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Midtown based high tech firm has an immediate ...,DevOps Engineer,"Configuration Management, Developer, Linux, Ma...",devops,NaN,NaN,ui
...,...,...,...,...,...,...,...,...
998,https://www.dice.com/jobs/detail/Software-Arch...,Global financial firm whose sole business is i...,Software Architect for Elite Hedge Fund,"Software Architect, .Net, Cloud",architect,NaN,NaN,NaN
999,https://www.dice.com/jobs/detail/Principal-Eng...,"Principal EngineerLocation: Boston, MA About Y...",Principal Engineer,"Agile, Analysis, Apache, Architecture, CASE, D...",NaN,NaN,NaN,NaN
1000,https://www.dice.com/jobs/detail/Oracle-Financ...,Position: Oracle Financial Specialist Locati...,Oracle Financial Specialist,"Oracle Financial,EBS,Oracle EBS,Oracle",NaN,NaN,NaN,NaN
1001,https://www.dice.com/jobs/detail/Strategy-Cons...,Are you passionate about delivering goals to y...,Strategy Consulting - Human Capital - HR Share...,"Consulting, Development, Excel, HTML, HTTP, Ma...",NaN,NaN,NaN,NaN


In [292]:
# Data Creation
tech_training = pd.DataFrame(tech_data[['jobtitle', 'jobdescription', 'tag']])

tech_training = tech_training.dropna()
tech_training = tech_training.reindex()


In [293]:
# Data Preprocessing
tech_training = tech_training.applymap(lambda x: x.lower() if isinstance(x, str) else x)
tech_training

old_word = r'\bit\b'

# This is to avoid the stopwords function from removing It since it reads it as "it"
tech_training['tag'] = tech_training['tag'].str.replace(old_word, 'information_technology', regex=True)



In [294]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Removes special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-Z\s]'  
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# remove whitespace
def remove_whitespace(text):
    return " ".join(text.split())


# remove keyfault stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

# Stemming 

stemmer = PorterStemmer()

def stem_words(text):
    word_tokens = word_tokenize(text)
    stems = [stemmer.stem(word) for word in word_tokens]
    return stems

# Lemmatization
lemmatizer = WordNetLemmatizer

def lemma_words(text):
    word_tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
    return lemmatized_words

tech_training = tech_training.applymap(lambda x: remove_special_characters(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: remove_punctuation (x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: remove_whitespace(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: remove_stopwords(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: stem_words(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: lemma_words(x) if isinstance(x, str) else x)




In [295]:
# Combining both job title and job description
tech_training['combined'] = tech_training['jobtitle'] + tech_training['jobdescription']
X = tech_training['combined'].apply(lambda x:x[0])




y = tech_training['tag'].apply(lambda x: x[0])


# Train Test Split (80% training data, 20% testing data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [296]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize 
tfidf_vectorizer = TfidfVectorizer()

# Fit transform the vectorizer

X_train_Tfidf_df = tfidf_vectorizer.fit_transform(X_train).toarray()

X_train_Tfidf_df = pd.DataFrame(X_train_Tfidf_df)

X_test_Tfidf_df = tfidf_vectorizer.transform(X_test).toarray()

X_test_Tfidf_df = pd.DataFrame(X_test_Tfidf_df)

tfidf_vectorizer.get_feature_names_out()




array(['android', 'angular', 'application', 'architect', 'assistant',
       'automated', 'automation', 'automationcontrol', 'aws', 'azure',
       'ba', 'back', 'backend', 'big', 'bigdata', 'bpm', 'build',
       'business', 'businessdata', 'cc', 'cloud', 'cognos', 'compliance',
       'contract', 'cyber', 'cybersecurity', 'data', 'database', 'dev',
       'developer', 'devops', 'director', 'disaster', 'domestic', 'dot',
       'eclinicalworks', 'electrical', 'embedded', 'enterprise',
       'facility', 'fireeeye', 'front', 'frontend', 'frontendui', 'full',
       'generalist', 'gui', 'help', 'hybrid', 'iam', 'implementation',
       'information', 'infrastructure', 'ios', 'java', 'javascript',
       'junior', 'kony', 'labview', 'lead', 'leadsr', 'linux', 'machine',
       'macintoshmac', 'magento', 'mainframe', 'manager', 'manufacturing',
       'mechanical', 'microservices', 'midlevel', 'mobile', 'multiple',
       'mysql', 'net', 'netc', 'network', 'odi', 'ods', 'ops', 'oracle',
 

In [297]:
model = GaussianNB()

model.fit(X_train_Tfidf_df, y_train)
print(model)

GaussianNB()


In [298]:
y_pred = model.predict(X_test_Tfidf_df)

In [299]:
accuracy_score(y_test, y_pred)


0.2755102040816326